<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [3]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [7]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [5]:
word_original = 'dog'
wv_embeddings.most_similar(positive=word_original, topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

Слово "cat" не входит в ближайшие слова 

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [8]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Anna\AppData\Local\Temp\ipykernel_12312\2760772041.py:9: SyntaxWarning: invalid escape sequence '\w'
  return re.findall('\w+', text)


In [9]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    embeddings_sentence = []
    words = tokenizer.tokenize(question)
    for w in words:
        if w in embeddings:
            embeddings_sentence.append(embeddings[w])
        else:
            continue
    if not embeddings_sentence: #Список пуст
        return np.zeros(dim)
    else:
        return np.mean(embeddings_sentence, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [5]:
question = 'I love neural networks'
emb = question_to_vec(question, wv_embeddings, tokenizer, dim=200)
print(round(emb[2], 2))

-1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: 1

Максимальное значение разницы между двумя функциями достигается, когда:

Hits@47 = 1 (хотя бы один релевантный элемент попадает в топ-47)
DCG@1 = 0 (релевантный элемент не находится на первой позиции)
В этом случае, максимум разницы равен 1 - 0 = 1. 

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

Ответ: [K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}$

In [53]:
import math
print(round(1 / math.log(10, 2), 1))

0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [10]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    sub_sum = 0
    for dup_rank in dup_ranks:
        if dup_rank <= k:
            sub_sum += 1
        else:
            sub_sum += 0
    return sub_sum / len(dup_ranks)
 

In [11]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    sub_sum = 0
    for dup_rank in dup_ranks:
        if dup_rank <= k:
            sub = 1 / np.log2(1 + dup_rank)
        else:
            sub = 0
        sub_sum += sub
    return sub_sum / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [6]:
import pandas as pd

In [7]:
def cosine_sim(v1, v2):
    # v1, v2 (200,)
    similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.nan_to_num(similarity, nan=0)  # replace NaN with 0

In [10]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
emb_origin = question_to_vec(copy_answers[0], wv_embeddings, tokenizer, dim=200)
similrity_rank = []
for i in candidates_ranking[0]:
    emb_copy = question_to_vec(i, wv_embeddings, tokenizer, dim=200)
    similrity_rank.append(cosine_sim(emb_origin, emb_copy))

sim_ind = similrity_rank.index(max(similrity_rank))
print('sim_ind', sim_ind, 'with similarity', max(similrity_rank))
#дубликат был первым в ранжированном списке
dup_ranks = [sim_ind + 1]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

sim_ind 1 with similarity 0.99999994
Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


C:\Users\Anna\AppData\Local\Temp\ipykernel_9252\704737424.py:3: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


У вас должно получиться

In [11]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

Считайте данные.

In [12]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [13]:
validation_data = read_corpus(r'E:\Iliushina_files\DLS\data\validation.tsv')

Кол-во строк

In [107]:
len(validation_data)

3760

Размер нескольких первых строк

In [108]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    pairs = []
    emb_question = question_to_vec(question, embeddings, tokenizer, dim)
    for i, candidate in enumerate(candidates):
        emb_cand = question_to_vec(candidate, embeddings, tokenizer, dim)
        pairs.append((i, candidate, cosine_similarity([emb_question], [emb_cand])[0][0]))
    return [pair[0:2] for pair in sorted(pairs, key=lambda x: x[2], reverse=True)]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [10]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [13]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  
Ответ : 1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [15]:
from tqdm import tqdm
from nltk.tokenize import WordPunctTokenizer

In [13]:
wv_ranking = []
max_validation_examples = 1000
for tokenizer in [MyTokenizer(), WordPunctTokenizer()]:
    print(tokenizer)
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print('---------------')

100%|██████████| 6/6 [00:00<00:00, 461.39it/s]


DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000
---------------
WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)


100%|██████████| 6/6 [00:00<00:00, 999.71it/s]

DCG@   1: 0.281 | Hits@   1: 0.281
DCG@   5: 0.337 | Hits@   5: 0.390
DCG@  10: 0.354 | Hits@  10: 0.444
DCG@ 100: 0.400 | Hits@ 100: 0.675
DCG@ 500: 0.426 | Hits@ 500: 0.877
DCG@1000: 0.439 | Hits@1000: 1.000
---------------


### Эмбеддинги, обученные на корпусе похожих вопросов

In [16]:
train_data = read_corpus(r'E:\Iliushina_files\DLS\data\train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [16]:
from gensim.models import Word2Vec
proc_words = [tokenizer.tokenize(text) for text in list(zip(*train_data))[0] + list(zip(*train_data))[1]]

Нам важно определить контекстуальную информацию, а не локальную, то бОльшие значения window нам подходят больше. Аналогично с min_count. Нужно игнорировать редкие слова и сосредоточиться на более частых ключевых словах, поэтому min_count 

In [36]:
import time

for min_count in [1, 3,6,10]:
    for window in [1,3,6,10]:
        print('min_count:', min_count, ', window:', window)
        current_time = time.time()
        embeddings_trained = Word2Vec(proc_words, # data for model to train on
                        vector_size=200,                 # embedding vector size
                        min_count=min_count,             # consider words that occured at least 5 times
                        workers=4,
                        window=window,
                        sg=1).wv
        end_time = time.time()
        print(end_time-current_time)
        wv_ranking = []
        max_validation_examples = 1000
        tokenizer = MyTokenizer()
        for i, line in enumerate(tqdm(validation_data)):
            if i == max_validation_examples:
                break
            q, *ex = line
            ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
            wv_ranking.append([r[0] for r in ranks].index(0) + 1)

        for k in tqdm([1, 5, 10, 100, 500, 1000]):
            print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
        print('------------------------')

min_count: 1 , window: 1
31.425585985183716


100%|██████████| 6/6 [00:00<00:00, 2000.46it/s]


DCG@   1: 0.272 | Hits@   1: 0.272
DCG@   5: 0.340 | Hits@   5: 0.403
DCG@  10: 0.358 | Hits@  10: 0.457
DCG@ 100: 0.403 | Hits@ 100: 0.677
DCG@ 500: 0.429 | Hits@ 500: 0.880
DCG@1000: 0.441 | Hits@1000: 1.000
------------------------
min_count: 1 , window: 3
51.01634740829468


100%|██████████| 6/6 [00:00<00:00, 1999.03it/s]


DCG@   1: 0.310 | Hits@   1: 0.310
DCG@   5: 0.389 | Hits@   5: 0.460
DCG@  10: 0.407 | Hits@  10: 0.514
DCG@ 100: 0.450 | Hits@ 100: 0.729
DCG@ 500: 0.473 | Hits@ 500: 0.908
DCG@1000: 0.483 | Hits@1000: 1.000
------------------------
min_count: 1 , window: 6
69.96009635925293


...

DCG@   1: 0.341 | Hits@   1: 0.341
DCG@   5: 0.419 | Hits@   5: 0.488
DCG@  10: 0.442 | Hits@  10: 0.560
DCG@ 100: 0.484 | Hits@ 100: 0.765
DCG@ 500: 0.507 | Hits@ 500: 0.940
DCG@1000: 0.513 | Hits@1000: 1.000
------------------------
min_count: 10 , window: 6
53.8204026222229


100%|██████████| 6/6 [00:00<00:00, 1499.93it/s]


DCG@   1: 0.353 | Hits@   1: 0.353
DCG@   5: 0.443 | Hits@   5: 0.520
DCG@  10: 0.463 | Hits@  10: 0.582
DCG@ 100: 0.508 | Hits@ 100: 0.797
DCG@ 500: 0.528 | Hits@ 500: 0.955
DCG@1000: 0.533 | Hits@1000: 1.000
------------------------
min_count: 10 , window: 10
67.7125928401947


100%|██████████| 6/6 [00:00<00:00, 1999.67it/s]

DCG@   1: 0.372 | Hits@   1: 0.372
DCG@   5: 0.454 | Hits@   5: 0.526
DCG@  10: 0.478 | Hits@  10: 0.600
DCG@ 100: 0.521 | Hits@ 100: 0.813
DCG@ 500: 0.539 | Hits@ 500: 0.954
DCG@1000: 0.544 | Hits@1000: 1.000
------------------------


чем больше window тем лучше, доказано результатами, хоть и занимает больше времени(70сек вместо 20-40сек). При переборе параметров, наибольшее качество было достигнуто при min_count: 10 , window: 10

#### Далее изучим как ведут себя различные токенизаторы при выбранных гиперпараметрах

##### TreebankWordTokenizer

In [18]:
from nltk.tokenize import TreebankWordTokenizer

current_time = time.time()
tokenizer = TreebankWordTokenizer()

proc_words = [tokenizer.tokenize(text) for text in list(zip(*train_data))[0] + list(zip(*train_data))[1]]
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                        vector_size=200,                 # embedding vector size
                        min_count=10,             # consider words that occured at least 5 times
                        workers=4,
                        window=10,
                        sg=1).wv
end_time = time.time()
print('time for model train:', end_time-current_time, ' sec')
wv_ranking = []
max_validation_examples = 1000

for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 1499.75it/s]

DCG@   1: 0.332 | Hits@   1: 0.332
DCG@   5: 0.417 | Hits@   5: 0.490
DCG@  10: 0.438 | Hits@  10: 0.554
DCG@ 100: 0.481 | Hits@ 100: 0.766
DCG@ 500: 0.504 | Hits@ 500: 0.940
DCG@1000: 0.510 | Hits@1000: 1.000


##### WordPunctTokenizer

In [13]:
from nltk.tokenize import WordPunctTokenizer
import time
from gensim.models import Word2Vec

current_time = time.time()
tokenizer = WordPunctTokenizer()

proc_words = [tokenizer.tokenize(text) for text in list(zip(*train_data))[0] + list(zip(*train_data))[1]]
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                        vector_size=200,                 # embedding vector size
                        min_count=10,             # consider words that occured at least 5 times
                        workers=4,
                        window=10,
                        sg=1).wv
end_time = time.time()
print('time for model train:', end_time-current_time, ' sec')
wv_ranking = []
max_validation_examples = 1000

for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

time for model train: 81.41124892234802  sec


100%|██████████| 6/6 [00:00<00:00, 1499.84it/s]

DCG@   1: 0.352 | Hits@   1: 0.352
DCG@   5: 0.442 | Hits@   5: 0.520
DCG@  10: 0.463 | Hits@  10: 0.585
DCG@ 100: 0.504 | Hits@ 100: 0.786
DCG@ 500: 0.525 | Hits@ 500: 0.947
DCG@1000: 0.530 | Hits@1000: 1.000


лучше всего работает простой токенайзер

#### с нормализацией

In [50]:
couple_list = [couple[0] + str(' ') + couple[1] for couple in train_data]

##### Spacy Lemmatizer

In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")
words_lemma = [[token.lemma_ for token in nlp(text)]
              for text in tqdm(couple_list)]
#слишком долго, обработка около 2,5 часов

##### nltk Lemmatizer

In [64]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
words_lemma = [[lemmatizer.lemmatize(word) for word in MyTokenizer().tokenize(text)]
              for text in tqdm(couple_list)]
#это работает меньше чем за минуту

100%|██████████| 1000000/1000000 [00:30<00:00, 32405.50it/s]


In [67]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words_lemma, # data for model to train on
                        vector_size=200,                 # embedding vector size
                        min_count=10,             # consider words that occured at least 5 times
                        workers=4,
                        window=10,
                        sg=1).wv

In [69]:
wv_ranking = []
max_validation_examples = 1000

for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, MyTokenizer())
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 1998.87it/s]

DCG@   1: 0.409 | Hits@   1: 0.409
DCG@   5: 0.491 | Hits@   5: 0.566
DCG@  10: 0.513 | Hits@  10: 0.634
DCG@ 100: 0.551 | Hits@ 100: 0.827
DCG@ 500: 0.568 | Hits@ 500: 0.953
DCG@1000: 0.573 | Hits@1000: 1.000


##### pymorphy3 Lemmatizer

In [75]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()
words_lemma_morphy3 = [[morph.parse(word)[0].normal_form for word in MyTokenizer().tokenize(text)]
              for text in tqdm(couple_list)]
#сработало за 2 минуты

100%|██████████| 1000000/1000000 [02:00<00:00, 8330.97it/s]


In [77]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words_lemma_morphy3, # data for model to train on
                        vector_size=200,                 # embedding vector size
                        min_count=10,             # consider words that occured at least 5 times
                        workers=4,
                        window=10,
                        sg=1).wv

In [78]:
wv_ranking = []
max_validation_examples = 1000

for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, MyTokenizer())
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 1999.35it/s]

DCG@   1: 0.264 | Hits@   1: 0.264
DCG@   5: 0.325 | Hits@   5: 0.380
DCG@  10: 0.341 | Hits@  10: 0.430
DCG@ 100: 0.382 | Hits@ 100: 0.631
DCG@ 500: 0.410 | Hits@ 500: 0.849
DCG@1000: 0.425 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:



* Простой токенайзер показал наилучшую точность. MyTokenizer (DCG@1000: 0.544) TreebankWordTokenizer(0.510) WordPunctTokenizer(0.530). Вероятнее всего это из-за того, что WordPunctTokenizer учитывает знаки препинания и регистр, а TreebankWordTokenizer еще сложнее и разбиваниет сокращения, притяжательные местоимения и дефисные слова на отдельные слова.
* Нормализация слов улучшила результат: DCG@1000: 0.573 при nltk.WordNetLemmatizer и MyTokenizer. При этом и обучение Word2Vec стало значительно дольше(2 минуты). Тестирование Spacy Lemmatizer показало, что его использование занимает сликом много времени на обработку слов (2,5 часа). 
* Эмбединги, обученные на корпусе вопроса, позволяют достичь большей точности, так как учитвают контекст вопроса (DCG@1000: 0.544 vs 0.444). Также было вывлено, что архитектура типа Skip-gram(sg=1) где контекстные слова предсказываются с использованием базового слова дает лучшее качество обучения, чем CBoW (результаты не сохранились, но этот параметр проверялся). Наилучшими для тренировки Word2Vec оказались эмбединги с min_count=10, window=10, потому что это позволяет оценивать весь контекст предложения. Гипер-параметр min_count не так сильно влиял на качество по среванению с гипер-параметром window.
* Полученные DCG Scores являются хорошими и модель способна эфективно ранжировать большинство случаев, однако в случае сложных кейсов часто ошибается. Для улучшения качества решения задачи можно использовать более современные модели. Также мера косинусного расстояния может быть не подходящей для этой задачи, можно потестировать Manhattan distance
* Можно побольше потестировать различные токенизаторы и убрать стоп-слова или слова паразиты